In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import random

2022-10-30 16:22:47.613908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.random.set_seed(13)
random.seed(13)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [4]:
df = pd.read_csv('../src/Fex_c.csv')

In [5]:
def read_dataframe(dataframe):
    dfc = dataframe.copy(True)
    dfc.drop('ImagesName', inplace=True, axis=1)
    return dfc

In [6]:
dfc = read_dataframe(df)

In [7]:
mean = dfc[dfc['RDR'] != 2]['RDR'].mean()

In [8]:
dfc.loc[dfc["RDR"] == 2.0, "RDR"] = mean

In [9]:
dfc.loc[dfc["RDR"] == 2.0]

,CDR,VCDR,RDR,I,S,N,T,Labels


In [10]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(7, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=50, verbose=0)

2022-10-30 16:22:53.136609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
predictions = model.predict(X_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

2/2 [==============================] - 0s 2ms/step


In [14]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.79
Precision: 0.77
Recall: 0.71


In [15]:
dfc = read_dataframe(df)

In [16]:
mean = dfc[dfc['RDR'] != 2]['RDR'].mean()

In [17]:
dfc.loc[dfc["RDR"] == 2.0, "RDR"] = mean

In [18]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [19]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
clf = svm.SVC()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

In [21]:
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
print(f'Precision: {precision_score(y_test, y_pred):.2f}')
print(f'Recall: {recall_score(y_test, y_pred):.2f}')

Accuracy: 0.68
Precision: 0.58
Recall: 0.79
